# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports

import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [ ]:
# here is the question; type over this to ask something new

system_prompt = """ You are a personal tutor for training the user about anything related to computer programming.
Please reply in a manner that the user understands the answer and it helps them build their understanding in that subject.
Provide examples wherever necessary. Use the tone of a helpful kind tutor who genuinely wants their student to understand
and improve in the subject.
"""

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
def stream_answer_from_gpt():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages = [
            {"role": "system", "content":system_prompt},
            {"role":"user", "content":question}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
# Get Llama 3.2 to answer

def stream_answer_from_llama():

    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    stream = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages = [
            {"role": "system", "content":system_prompt},
            {"role":"user", "content":question}
        ],
    stream = True)
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)



In [21]:
stream_answer_from_llama()

I see you're working with Python, specifically with a concept called **iterators**.

Let's break it down step by step:

1. `yield` is a Python keyword that allows us to create generators (functions that produce iterables).

2. You have an expression like `{book.get("author") for book in books if book.get("author")}` inside the parentheses.
   - This part of the code is using **database query** style similar to SQL `SELECT * FROM table WHERE column = value`, but with a list comprehension.

3. But notice we don't even need this expression, and that's because we've got an extra keyword before it: `yield from`.
  
So let's simplify our original piece of code:
   python
yield from {book.get("author") for book in books if book.get("author")}

    Can be broken down as:

   
for each_book in {book for book in books if book.get("author")} : 
  yield each_book["author"].get()


But our simplified version is

   python
yield from {each_book['author'] for each_book in books if each_book.get("author")}

Now you have a generator `yield` that goes over items of "books" collection only when they have an author. We want to yield the authors.

You might be wondering how we can achieve this. Well here’s a code sample:

python
import json

class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author

# Create some books with varying 'author' values.
books = [
    {"title": "Book 1", "author": ""},
    {"title": "Book 2", "author": "Author 1"},
    {"title": "Book 3", "author": None}, # Optional value so it would pass the filter
    {"title": "Book 4", "author": ''}
]

def generate_authors(books):
   """
   Iterate of books in `books` where `"author"` exists and yielding `"author"`.
   So you only yield one item (one book), but the generator works exactly like a for:
   Each iteration yields one value for each book which matches your conditions.
   
   Args:       - books : Collection of Books that would return authors from their objects 
                  when calling function generate_authors()
   Returns:    - list of `books['author']
                    for book in books if book["author"]
       """
    yield from {book['author'] for book in books if book.get("author")} # yields all valid authors
    
# Create an instance of the Book class
book1 = Book('Book 1', 'Author 1')
book2 = Book('Book 2', 'Author 2')

books.append(book1) # Add to collection
books.append(book2) # Adds another book before appending last one which does yield 'None' (we used if condition)

for author in generate_authors(books): 
   if author is not None:
       print(author)


In the example, we're working with an item that has `author` but it can be null. If you want all valid authors so they aren't `None`, you have to use `if book['author'] is not None or book.get('author')`.